In [8]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc,Dash
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Imports for plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animalShelter import AnimalShelter
import base64


###########################
# Data Manipulation / Model
###########################
# Log into AnimalShelter Database
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)

# Creates DataFrame from records in AnimalShelter Database
df = pd.DataFrame.from_records(shelter.read())
df = df.drop(columns=['_id'])

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################

# Generates JupyterDash app
app = JupyterDash(__name__)

#Image encoding for use in HTML
image_filename = '/home/alexandercasa_snhu/Desktop/Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Layout for HTML app
app.layout = html.Div([
                html.Center(html.Img( src='data:image/png;base64,{}'.format(encoded_image.decode()))),
                html.Center(html.B(html.H1('Alex Casanova CS-340 Dashboard'))),
                html.Hr(),
                html.Div(dcc.RadioItems(id='filter',
                    options=[
                        {'label': 'Water Rescue', 'value': 'Water Rescue'},
                        {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
                        {'label': 'Disaster Rescue/Individual Tracking', 'value': 'Disaster Rescue/Individual Tracking'},
                        {'label': 'Reset', 'value': 'Reset'}
                    ],
                    value='Reset',
                    labelStyle={'display': 'inline-block'}
                    )),
                dash_table.DataTable(id='datatable-id',
                    columns=[
                        {"name": i, "id": i} for i in df.columns
                     ],
                    data=df.to_dict('records'),
                                     
                    # HTML controls for DataFrame to make it more user-friendly
                    editable=False,
                    filter_action="native",
                    sort_action="native",
                    sort_mode="multi",
                    column_selectable=False,
                    row_selectable = "single",
                    row_deletable=False,
                    selected_columns=[],
                    selected_rows=[0],
                    page_action="native",
                    page_current=0,
                    page_size=10
                ), 
                html.Br(),
                html.Hr(),
                html.Div(className='row',
                    style={'display' : 'flex'},
                    children=[
                        html.Div(id='map-id',  className='col s12 m6'),
                        html.Div(id='graph-id',className='col s12 m6'),
                    ])
        ])
        
        

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'id': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback([Output('datatable-id','data'),
              [Input('filter', 'value')]])

# Updates Dashboard based on filter radio buttons above
def update_dashboard(filter_type):
        if filter_type == 'Water Rescue':
            df = pd.DataFrame.from_records(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                          {'$or': [
                                                              {'breed': 'Labrador Retriever Mix'},
                                                              {'breed': 'Chesa Bay Retr Mix'},
                                                              {'breed': 'Newfoundland'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                }))
        elif filter_type == 'Mountain/Wilderness Rescue':
            df = pd.DataFrame.from_records(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Alaskan Malamute'},
                                                              {'breed': 'Old English Sheepdog'},
                                                              {'breed': 'Rottweiler'},
                                                              {'breed': 'Siberian Husky'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                }))
        elif filter_type == 'Disaster Rescue/Individual Tracking':
            df = pd.DataFrame.from_records(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'Doberman Pinscher'},
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Golden Retriever'},
                                                              {'breed': 'Bloodhound'},
                                                              {'breed': 'Rottweiler'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                          }]
                                                }))
        elif filter_type == 'Reset':
            df = pd.DataFrame.from_records(shelter.read({}))

        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        df = df.drop(columns=['_id'])
        data=df.to_dict('records')
        
        return (data)

# Updates Pie Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_records(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    return dcc.Graph(figure = px.pie(data_frame=dff, values = values, names = names, width=1000,height=500))

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'selected_columns')])
# Updates Map based on DataFrame selection
def update_map(viewData, selected_rows, selected_columns):
    dff = pd.DataFrame.from_dict(viewData)
    return dl.Map(style={'width':'1000px', 'height': '500px'}, 
                  center=[30.75,-97.48], 
                  zoom=10, 
                  children=[
                      dl.TileLayer(id="base-layer-id"),
                      dl.Marker(position=[(dff.iloc[selected_rows[0],13]), 
                                          (dff.iloc[selected_rows[0],14])], 
                                children=[dl.Tooltip(dff.iloc[selected_rows[0],4])]
                               )
                  ])

# Starts the Dash app
app.run_server()

Dash app running on http://127.0.0.1:22597/
